In [ ]:
# !sudo python3 -m pip install -U slackclient

In [ ]:
# !sudo apt-get install python3-mysqldb

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

import time


In [ ]:
import json
from slackclient import SlackClient

# Read the access token from the file
secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)
token = auth_info["access_token"]
sc = SlackClient(token)


In [ ]:
# Parameters to connect to MySQL database
parameters = {
    'host':     'localhost', 
    'username': 'root', 
    'password': 'dwdstudent2015', 
    'database': 'slack',
    'port':     '3306',
    'charset':  'utf8',
    'use_unicode': True
}

In [ ]:
import mysql.connector
from sqlalchemy import create_engine


template = 'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}?charset={charset}'
conn_string = template.format(**parameters)

engine = create_engine(conn_string, echo=False)


In [ ]:
df = pd.read_sql("SELECT * FROM messages", con=engine)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)

In [ ]:
baseurl = "http://ipython-panos.ipeirotis.com:5000/"

In [ ]:
# Generate channel sentiment reports

df_channels = pd.read_sql("SELECT * FROM channels", con=engine)
df_channels.set_index('id', inplace=True)

attachments = []

for i, row in df_channels[["name"]].iterrows():
    ts = str(time.time())
    channel_id = i
    channel_name = row["name"]
    plot_df = df[ df['channel_id'] == channel_id].sort_index()
    if len(plot_df)>2000:
        ax = plot_df.rolling(window=300).mean()[['pos', 'neg']].plot(figsize=(10,6), ylim=(0,0.25))
        ax.set_xlabel("Sentiment on #"+channel_name)
        fig = ax.get_figure()
        filename = 'plots/'+channel_name+'-'+ts+'.png'
        fig.savefig(filename)
        attachment = {
            "image_url": baseurl + filename,
            "text": "Sentiment on channel #" + channel_name
        }
        attachments.append(attachment)
        
sc.api_call("chat.postMessage", channel="#bot_stats", text="Channel Sentiment Report", attachments=attachments)

In [ ]:
#user_stats = df.groupby(df['user']).mean()

#data = df_users[['profile.real_name']].join(user_stats)
#data

In [ ]:
# Generate user sentiment reports

df_users = pd.read_sql("SELECT * FROM users WHERE deleted=False AND is_bot=False", con=engine)
df_users.set_index('id', inplace=True)

attachments = []

for i, row in df_users[['profile.real_name']].iterrows():
    ts = str(time.time())
    user_id = i
    user_name = row["profile.real_name"]
    plot_df = df[ (df['user'] == user_id)].sort_index()
    plot_df["diff"] = plot_df['pos'] - plot_df['neg']
    plot_df["sentiment"] = plot_df["diff"].rolling(center=False, win_type='parzen', window=100).mean()
    # plot_df = plot_df.rolling(window=200, win_type='parzen').mean()
    
    if len(plot_df) == 0:
        continue
    ax = plot_df[['sentiment']].cumsum().plot(figsize=(10,6))
    ax.set_xlabel("Sentiment of user "+user_name)
    ax.grid(True)
    fig = ax.get_figure()
    filename = 'plots/'+user_id+'-'+ts+'.png'
    fig.savefig(filename)
    attachment = {
        "image_url": baseurl + filename,
        "text": "Sentiment of user " + user_name
    }
    attachments.append(attachment)

# sc.api_call("chat.postMessage", channel="#bot_stats", text="User Sentiment Report", attachments=attachments)
    

In [ ]:
df_users['profile.real_name']

In [ ]:
df_users.index.values

In [ ]:
data = []

# for i, row in df_users[['profile.real_name']].iterrows()

user_id = 'U0HLS92BX'
user_name = 'Matthew Voshell'

print(user_name)
for j, row in df[['user','message']].iterrows():
    user = row['user']
    if user == user_id:
        tag = 'pos'
    else:
        tag = 'neg'
    data.append( (tag, row['message']) )

data

In [ ]:
# This is the number of positive sentences
len([tag for (tag, s) in data if tag=='pos'])

In [ ]:
# This is the number of negative sentences
len([tag for (tag, s) in data if tag=='neg'])

In [ ]:
import nltk

def features(msg):
    if msg==None or len(msg)==0 or pd.isnull(msg):
        return {}
    sent_tokens = nltk.word_tokenize(msg)
    # document_words = nltk.pos_tag(sent_tokens)
    features = {}
    #for word, tag in document_words:
    for word in sent_tokens:
        if word in df_users.index.values:
            continue
        if '|' in word:
            continue
        #if tag.startswith('NNP'):
        #features['%s/%s' % (word, tag)] = True
        features[word] = True
    return features

labeled_features = [(features(sent), tag) for (tag, sent) in data]

In [ ]:
import random

test_set_size = int(len(labeled_features)/8)
random.shuffle(labeled_features)
train_set, test_set = labeled_features[test_set_size:], labeled_features[:test_set_size]


In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
msg = classifier.show_most_informative_features(100)

In [ ]:
m = '''
                Material = True              pos : neg    =    252.5 : 1.0
                  pinned = True              pos : neg    =    199.3 : 1.0
                 `Today` = True              pos : neg    =    191.9 : 1.0
                     Yea = True              pos : neg    =    177.4 : 1.0
                   Image = True              pos : neg    =    171.7 : 1.0
                  Design = True              pos : neg    =    165.5 : 1.0
       utm_medium=social = True              pos : neg    =    151.5 : 1.0
           //medium.com/ = True              pos : neg    =    151.5 : 1.0
                     yea = True              pos : neg    =    133.7 : 1.0
              Foundation = True              pos : neg    =    131.3 : 1.0
                 YouTube = True              pos : neg    =    131.3 : 1.0
     utm_campaign=buffer = True              pos : neg    =    131.3 : 1.0
                  worthy = True              pos : neg    =    111.1 : 1.0
                invision = True              pos : neg    =    111.1 : 1.0
                   now.. = True              pos : neg    =    111.1 : 1.0
                     biz = True              pos : neg    =    111.1 : 1.0
                  sketch = True              pos : neg    =    103.0 : 1.0
                   whois = True              pos : neg    =     90.9 : 1.0
                    York = True              pos : neg    =     90.9 : 1.0
//i.giphy.com/Q56SF4czEtSZG.gif = True              pos : neg    =     90.9 : 1.0
                 sprints = True              pos : neg    =     90.9 : 1.0
  utm_source=twitter.com = True              pos : neg    =     90.9 : 1.0
                 Channel = True              pos : neg    =     90.9 : 1.0
                      ha = True              pos : neg    =     81.9 : 1.0
                  lookup = True              pos : neg    =     78.8 : 1.0
                  walked = True              pos : neg    =     70.7 : 1.0
                  time.. = True              pos : neg    =     70.7 : 1.0
              Experience = True              pos : neg    =     70.7 : 1.0
                personas = True              pos : neg    =     70.7 : 1.0
                   staff = True              pos : neg    =     70.7 : 1.0
'''

In [ ]:
sc.api_call("chat.postMessage", channel="#bot_stats", text="@matthew characteristic phrases:\n```"+m+"```")